<a href="https://colab.research.google.com/github/adilabduakhanov/Developing_Neuro-Employees_on_GPT/blob/main/%D0%A2%D0%B5%D0%BB%D0%B5%D0%B3%D1%80%D0%B0%D0%BC_%D0%B1%D0%BE%D1%82_%D0%BD%D0%B0_Aiogram3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================
# УСТАНОВКА БИБЛИОТЕК
# ==============================
# !pip install aiogram==3.13.1 openai nest_asyncio

import asyncio
import logging
import nest_asyncio
from aiogram import Bot, Dispatcher, F
from aiogram.types import Message
from aiogram.filters import Command
from aiogram.client.default import DefaultBotProperties
from aiogram.enums import ParseMode
from openai import OpenAI
from google.colab import userdata

# Для Google Colab
nest_asyncio.apply()

# ==============================
# ТОКЕНЫ
# ==============================
TELEGRAM_TOKEN = userdata.get("TELEGRAM_TOKEN")
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

# ==============================
# ИНИЦИАЛИЗАЦИЯ GPT
# ==============================
client = OpenAI(api_key=OPENAI_API_KEY)

# ==============================
# SYSTEM PROMPT
# ==============================
SYSTEM_PROMPT = """
Ты — эксперт по дизайну интерьера маленьких квартир (до 50 м²).

Строгие правила:

1. Отвечаешь только по теме дизайна маленьких квартир.
2. Если вопрос не по теме — откажись в стихах.
3. Все ответы — в стихотворной форме.
4. Максимум 8 строк.
5. Без списков.
6. Без эмодзи.
7. Стиль — лёгкая рифма, напоминающая Пушкина.
8. Ответ должен быть практичным, но оформленным как стих.

Ты не обсуждаешь:
политику, медицину, инвестиции, технологии, программирование и любые другие темы.
"""

# ==============================
# ХРАНЕНИЕ ИСТОРИИ
# ==============================
user_dialogs = {}

# ==============================
# СОЗДАНИЕ БОТА
# ==============================
bot = Bot(
    token=TELEGRAM_TOKEN,
    default=DefaultBotProperties(parse_mode=ParseMode.HTML)
)

dp = Dispatcher()

# ==============================
# КОМАНДА /start
# ==============================
@dp.message(Command("start"))
async def start_handler(message: Message):
    await message.answer(
        "Привет.\n"
        "Я поэтический эксперт по дизайну маленьких квартир.\n\n"
        "Задай вопрос о планировке, цветах или мебели.\n"
        "Команда /clear — очистить историю."
    )

# ==============================
# КОМАНДА /clear
# ==============================
@dp.message(Command("clear"))
async def clear_handler(message: Message):
    user_id = message.from_user.id
    user_dialogs[user_id] = []
    await message.answer("История очищена. Начнём новый разговор о дизайне.")

# ==============================
# КОМАНДА /help
# ==============================
@dp.message(Command("help"))
async def help_handler(message: Message):
    await message.answer(
        "/start — запуск\n"
        "/clear — очистить историю\n"
        "/help — помощь\n\n"
        "Бот отвечает только по теме дизайна маленьких квартир и только стихами."
    )

# ==============================
# ОБРАБОТКА СООБЩЕНИЙ
# ==============================
@dp.message(F.text & ~F.text.startswith("/"))
async def message_handler(message: Message):
    user_id = message.from_user.id
    user_text = message.text

    # Если пользователь новый — создаём историю
    if user_id not in user_dialogs:
        user_dialogs[user_id] = []

    # Добавляем сообщение пользователя
    user_dialogs[user_id].append({
        "role": "user",
        "content": user_text
    })

    # Формируем сообщения для GPT
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT}
    ] + user_dialogs[user_id]

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature=0.8,
            max_tokens=300
        )

        answer = response.choices[0].message.content

        # Сохраняем ответ в историю
        user_dialogs[user_id].append({
            "role": "assistant",
            "content": answer
        })

        await message.answer(answer)

    except Exception as e:
        await message.answer("Произошла ошибка при обращении к GPT.")
        print("Ошибка:", e)

# ==============================
# ЗАПУСК БОТА
# ==============================
async def main():
    print("Бот запущен...")
    await dp.start_polling(bot)

if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)
    asyncio.run(main())


Бот запущен...
